# Proyecto 3: Predicción de precios de propiedades

¡Bienvenidos al tercer proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso armar nuestros primeros modelos para predecir el precio de las propiedades en dólares.

Las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadrático medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
print("El dataset que vamos a trabajar aquí tiene {} observaciones".format(df.shape[0]))

El dataset que vamos a trabajar aquí tiene 6376 observaciones


In [4]:
df.head(5)

,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,barrio_match,PH,apartment,house,...,VILLA LUGANO,VILLA LURO,VILLA ORTUZAR,VILLA PUEYRREDON,VILLA REAL,VILLA RIACHUELO,VILLA SANTA RITA,VILLA SOLDATI,VILLA URQUIZA,outlier_price_m2
0,-34.589,-58.417,170000.000,40.000,38.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-34.591,-58.418,90000.000,27.000,27.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,-34.587,-58.437,150000.000,44.000,44.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-34.593,-58.428,154000.000,58.000,58.000,2,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-34.593,-58.428,154000.000,58.000,58.000,3,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6376 entries, 0 to 6375
Data columns (total 59 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   lat                    6376 non-null   float64
 1   lon                    6376 non-null   float64
 2   price_aprox_usd        6376 non-null   float64
 3   surface_total_in_m2    6376 non-null   float64
 4   surface_covered_in_m2  6376 non-null   float64
 5   rooms                  6376 non-null   int64  
 6   barrio_match           6376 non-null   int64  
 7   PH                     6376 non-null   int64  
 8   apartment              6376 non-null   int64  
 9   house                  6376 non-null   int64  
 10  AGRONOMIA              6376 non-null   int64  
 11  ALMAGRO                6376 non-null   int64  
 12  BALVANERA              6376 non-null   int64  
 13  BARRACAS               6376 non-null   int64  
 14  BELGRANO               6376 non-null   int64  
 15  BOCA

In [8]:
df.outlier_price_m2.unique()

array([0], dtype=int64)

El objetivo de este proyecto es poder trabajar en el ajuste de modelos y su posterior evaluación.

Para empezar vamos a separar el `dataset` en un conjunto de entrenamiento (80%) y un conjunto de test (20%). 

**Separá el dataset** en `X_train`, `X_test`, `y_train` e `y_test` con el tamaño correspondiente

In [9]:
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = .2,random_state=42)


## Árboles de decisión

Lo primero que vamos a hacer es entrenar un árbol de decisión y usar de métrica al `RMSE`. 

Para poder obtener el **RMSE** vamos a medir el `mean_squared_error` y obtener su raíz cuadrada. 

**Importá** `DecisionTreeRegressor` desde `sklearn.tree`.  

A continuación **entrená** el regresor con el conjunto de training


In [ ]:
from sklearn.tree import DecisionTreeRegressor
reg_tree = DecisionTreeRegressor(criterion= 'mse',max_depth= 10,random_state=42)

In [ ]:
reg_tree.fit(X_train, y_train)

Con el modelo entrenado **realizá la predicción** sobre el conjunto de test `X_test` y guardá el resultado en una variable `y_pred`.

In [ ]:
y_pred = reg_tree.predict(X_test)

**Calculá el rmse** sacando la raíz cuadrada de `mean_squared_error` entre `y_test` e `y_pred` y **mostrá** el resultado

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
rmse

__Analizar el cambio en el RMSE a medida que es más profundo el árbol de decisión, tanto en training como en testing.__

Para esto, **iterá** de 5 en 5 en el parámetro `max_depth` y **observá** como impacta en el RMSE. 

**Creá** dos arreglos `rmses_train` y `rmses_test` para ir guardando los **rmse** de cada profundidad

In [ ]:
max_depth =np.arange(5,31,5)
max_depth

In [ ]:
rmses_train= np.array([])
rmses_test= np.array([])

In [ ]:

for m in max_depth:
    reg_tree = DecisionTreeRegressor(criterion= 'mse',max_depth= m,random_state=42)
    reg_tree.fit(X_train, y_train)
    y_pred_test = reg_tree.predict(X_test)
    y_pred_train = reg_tree.predict(X_train)
    rmses_train=np.append(rmses_train,np.sqrt(mean_squared_error(y_pred_train,y_train)))
    rmses_test=np.append(rmses_test,np.sqrt(mean_squared_error(y_pred_test,y_test)))    
    

In [ ]:
rmses_train

In [ ]:
rmses_test

Ahora graficamos los valores que guardamos en los arreglos `rmses_train` y `rmses_test`

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib inline 
plt.rcParams['figure.figsize'] = (8,6)
plt.plot(range(5,31, 5), rmses_train,'-,',marker = 'o', label='RMSE Training')
plt.plot(range(5,31, 5), rmses_test,'-,',marker = 'o', label='RMSE Testing')
plt.ylim((0, 30000))
plt.legend(loc="best")
plt.title("RMSE Training vs RMSE Testing para árboles de decisión")
plt.ylabel('RMSE')
plt.xlabel('Profundidad Arbol de Decisión')
plt.show()


Podemos ver aquí como el modelo presenta sobreajuste dado que a mayor complejidad (en este caso, mayor profundidad del árbol) más diferencia entre los resultados de training y testing. También observamos como la curva de testing decrece y luego vuelvo a crecer. El punto donde se minimiza está relacionado con el tradeoff entre sesgo y varianza que vamos a ver en la próxima unidad.

## KNN

**Entrená un knn** y nuevamente medir el **rmse** en el conjunto de testing


In [ ]:
print('Maximos: \n',pd.DataFrame(X_train).max(),'\n \n Mínimas: \n',pd.DataFrame(X_train).min(),'\n')
print('Means: \n',pd.DataFrame(X_train).mean(),'\n \n Std: \n',pd.DataFrame(X_train).std())

Lo ideal sería eliminar los outliers que aparecen. Hay propiedades con superficies nulas y otras con superficies muy grandes 120000m2 (12 hectareas!)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) 

In [ ]:
print('Maximos: \n',pd.DataFrame(X_train).max(),'\n \n Mínimas: \n',pd.DataFrame(X_train).min(),'\n')
print('Means: \n',pd.DataFrame(X_train).mean(),'\n \n Std: \n',pd.DataFrame(X_train).std())

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

reg_knn = KNeighborsRegressor(n_neighbors=10 )
reg_knn.fit(X_train,y_train)

y_train_pred = reg_knn.predict(X_train)
y_test_pred = reg_knn.predict(X_test)


In [ ]:
knn_rmses_test=np.sqrt(mean_squared_error(y_test,y_test_pred))
knn_rmses_test

__Analizar el cambio en el RMSE a medida que consideramos más vecinos para KNN, tanto en training como en testing.__

Para esto, **iterá** incrementando de a uno el parámetro `n_neighbors` y **observá** como impacta en el RMSE. 

**Creá** dos arreglos `rmses_train` y `rmses_test` para ir guardando los **rmse** de cada profundidad

In [ ]:
n_neighbors =np.arange(1,30,1)
n_neighbors

In [ ]:
rmses_train= np.array([])
rmses_test= np.array([])

In [ ]:
rmses_test

In [ ]:
for k in n_neighbors:
    
    reg_knn = KNeighborsRegressor(n_neighbors=k)
    
    # Entrenar el modelo
    reg_knn.fit(X_train,y_train)
    
    # Predecir y evaluar sobre el set de entrenamiento
    y_train_pred = reg_knn.predict(X_train)
    rmses_train=np.append(rmses_train,np.sqrt(mean_squared_error(y_train,y_train_pred)))
    
    
    # Predecir y evaluar sobre el set de evaluación
    y_test_pred = reg_knn.predict(X_test)
    rmses_test=np.append(rmses_test,np.sqrt(mean_squared_error(y_test,y_test_pred)))

In [ ]:
plt.rcParams['figure.figsize'] = (8,6)
plt.plot(range(1,30, 1), rmses_train,'-,',marker = 'o', label='RMSE Training')
plt.plot(range(1,30, 1), rmses_test,'-,',marker = 'o', label='RMSE Testing')
plt.xlim((0, 30))
plt.legend(loc="best")
plt.title("RMSE Training vs RMSE Testing para KNN")
plt.ylabel('RMSE')
plt.xlabel('Cantidad de vecinos')
plt.show()

**Calcular el RMSE promedio del resultado de cross validation para un árbol de decisión. 
Como parámetros deberás usar:**
- 10 folds
- profundidad 5
- scoring neg_mean_squared_error.

El árbol de decisión guardalo en una variable llamada `regressor` para poder reutilizarla luego.

Atención: `cross_validation_score` de `scikit-learn` usa la métrica `neg_mean_squared_error` (NMSE) en vez de `mean_square_error` (MSE). 

`NMSE` es lo mismo que `MSE` pero con un signo negativo. 

Como nosotros queremos obtener el `MSE` para calcular sobre él el `RMSE`, vamos a definir un método `nmsq2rmse` para hacer esa conversión de `NMSE` a `MSE`
.

In [ ]:
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = .2,random_state=42)


In [ ]:
regressor = DecisionTreeRegressor(criterion='mse' ,max_depth = 5,random_state=42)

In [ ]:
def nmsq2rmse(score):
    return np.sqrt(-score)

In [ ]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(regressor, X_train, y_train,
                             scoring="neg_mean_squared_error", cv=10)

In [ ]:
score

In [ ]:
rmse = nmsq2rmse(score)

In [ ]:
rmse

In [ ]:
RMSE_prom = rmse.mean()

In [ ]:
RMSE_prom

Para ver el resultado final, reentrenamos al regresor y mostramos en un dataframe la comparación entre los valores reales, los predichos y su diferencia 

In [ ]:
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_pred)

In [ ]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)'),(abs(val_real-val_pred)/val_real*100).rename('error_porcentual')] ,  axis=1)

In [ ]:
predicciones.head(10)

In [ ]:
sns.distplot(predicciones['error_porcentual'],bins = 12)


plt.title("Distribución de valores de error porcentual")
plt.ylabel('cantidad de valores')
plt.xlabel('error porcentual')
plt.show()